In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
import pickle
from statistics import mean

In [2]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer, MWETokenizer
from nltk.util import ngrams
import re
import string
from nltk.corpus import stopwords
from nltk.chunk import ne_chunk
set(stopwords.words('english'))
from nltk.tag import pos_tag
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

In [3]:
got_reddit = pickle.load( open( "./data/got_reddit.pkl", "rb" ) )
asoiaf_reddit = pickle.load( open( "./data/asoiaf_reddit.pkl", "rb" ) )
freefolk_reddit = pickle.load( open( "./data/freefolk_reddit.pkl", "rb" ) )

In [4]:
print(freefolk_reddit.shape)
print(asoiaf_reddit.shape)
print(got_reddit.shape)

(189774, 2)
(28807, 2)
(158665, 2)


In [5]:
freefolk_reddit['sentiment'] = ''
asoiaf_reddit['sentiment'] = ''
got_reddit['sentiment'] = ''

freefolk_reddit['sentiment_compound'] = ''
asoiaf_reddit['sentiment_compound'] = ''
got_reddit['sentiment_compound'] = ''

freefolk_reddit['sentiment_sum'] = ''
asoiaf_reddit['sentiment_sum'] = ''
got_reddit['sentiment_sum'] = ''

freefolk_reddit['sentiment_mean'] = ''
asoiaf_reddit['sentiment_mean'] = ''
got_reddit['sentiment_mean'] = ''

In [6]:
counts_free = freefolk_reddit['body'].value_counts()
freefolk_reddit = freefolk_reddit[~freefolk_reddit['body'].isin(counts_free[counts_free > 1].index)]

counts_a = asoiaf_reddit['body'].value_counts()
asoiaf_reddit = asoiaf_reddit[~asoiaf_reddit['body'].isin(counts_a[counts_a > 1].index)]

counts_g = got_reddit['body'].value_counts()
got_reddit = got_reddit[~got_reddit['body'].isin(counts_g[counts_g > 1].index)]

In [7]:
print(freefolk_reddit.shape)
print(asoiaf_reddit.shape)
print(got_reddit.shape)

(175822, 6)
(28280, 6)
(153179, 6)


In [8]:
def cleaning(text):
    got_stopwords = ['[spoilers]', '[deleted]', 'nan', '[removed]', '[spoiler]', 'deleted', '[Spoilers', 'Main]', 'Spoilers', 'Main', 'removed', 'spoiler', '[All', 'Spoilers]', 'all', 'spoilers', '[NO SPOILER]', 'no spoiler', '[SPOLER]', '(Spoilers', 'Main)', 'spoler', '[Spoilers]', '[SPOILERS]', 'SPOILERS', 'MAIN]', 'MAIN', 'extended)', 'extended', 'Production)', 'Production', 'EXTENDED]', 'EXTENDED', '[NO', 'SPOILERS]', 'NO', '*Spoilers', 'Ahead*', 'Ahead', 'Extended]', 'Extended', '{SPOILER}', 'SPOILER', '[No', 'Spoiler]', 'No', 'Spoiler', '(NO', 'SPOILERS)', 'MAIN)', '(SPOILERS', '(no', 'spoilers)', '[SPOILERS', 'Extended)']
    text_words = text.split()
    #print(text_words)
    result_words = [word for word in text_words if word not in got_stopwords]
    cleaned_text = ' '.join(result_words)
    #print(cleaned_text)
    return cleaned_text

In [10]:
def transformation(df):
    for i, j in df.iterrows(): 
            text = df['body'][i]
        #print(text)
            cleaned_text = cleaning(text)
        #print(cleaned_text)
            df['body'][i] = cleaned_text

In [11]:
transformation(got_reddit)
transformation(freefolk_reddit)
transformation(asoiaf_reddit)

In [12]:
#pickle.dump(freefolk_reddit, open("freefolk_reddit_after_removing_spam_monday.pkl", "wb" ) )
#pickle.dump(asoiaf_reddit, open("soiaf_reddit_after_removing_spam_monday.pkl", "wb" ) )
#pickle.dump(got_reddit, open("got_reddit_after_removing_spam_monday.pkl", "wb" ) )

In [13]:
sid = SentimentIntensityAnalyzer()

def sentiment(df):
    for i, j in df.iterrows(): 
        text = df['body'][i]
        sentences = tokenize.sent_tokenize(text)
        sentiment = []
        for sentence in sentences:
            ss = sid.polarity_scores(sentence)
            sentiment.append(ss)
        df['sentiment'][i] = sentiment

In [14]:
def sentiment_compound_func(df):
    for i, j in df.iterrows(): 
        text = df['sentiment'][i]
        sentiment_list = []
        for item in range(len(text)):
            sentiment = text[item]['compound']
            sentiment_list.append(sentiment)
            df['sentiment_compound'][i] = sentiment_list

In [15]:
def sentiment_sum_func(df):
    for i, j in df.iterrows(): 
        df['sentiment_sum'][i] = sum(df['sentiment_compound'][i])

In [18]:
def sentiment_mean_func(df):
    for i, j in df.iterrows(): 
        df['sentiment_mean'][i] = mean(list(df['sentiment_compound'][i]))

In [17]:
sentiment(freefolk_reddit)
sentiment(asoiaf_reddit)
sentiment(got_reddit)

sentiment_compound_func(freefolk_reddit)
sentiment_compound_func(asoiaf_reddit)
sentiment_compound_func(got_reddit)

sentiment_sum_func(freefolk_reddit)
sentiment_sum_func(asoiaf_reddit)
sentiment_sum_func(got_reddit)

pickle.dump(freefolk_reddit, open("freefolk_reddit_cleaned_monday.pkl", "wb" ) )
pickle.dump(asoiaf_reddit, open("soiaf_reddit_cleaned_monday.pkl", "wb" ) )
pickle.dump(got_reddit, open("got_reddit_cleaned_monday.pkl", "wb" ) )

In [20]:
got_reddit = pickle.load( open( "./data/got_reddit_cleaned_monday.pkl", "rb" ) )
asoiaf_reddit = pickle.load( open( "./data/soiaf_reddit_cleaned_monday.pkl", "rb" ) )
freefolk_reddit = pickle.load( open( "./data/freefolk_reddit_cleaned_monday.pkl", "rb" ) )